In [5]:
import pandas as pd


file_path = "SoundTracks/set1_tracklist.csv"

# Load CSV into DataFrame
df = pd.read_csv(file_path)
df

,Nro,Emotion,Album name,Track,Min:Sec
0,1,Happy,Shallow Grave,6,02:02-02:17
1,2,Happy,The Rainmaker,2,01:54-02:07
2,3,Happy,The Rainmaker,3,02:55-03:13
3,4,Happy,Man of Galilee CD1,2,03:02-03:18
4,5,Happy,Shakespeare in Love,1,01:06-01:22
...,...,...,...,...,...
355,356,Low Tens.,Lethal weapon 3,7,03:54-04:13
356,357,Low Tens.,Dracula,4,00:55-01:09
357,358,Low Tens.,Angel Heart,8,00:39-00:55
358,359,Low Tens.,Batman,18,02:46-03:02


In [6]:
df['Nro'] = df['Nro'].apply(lambda x: '{:03d}'.format(x))

In [7]:
df.head(10)

,Nro,Emotion,Album name,Track,Min:Sec
0,001,Happy,Shallow Grave,6,02:02-02:17
1,002,Happy,The Rainmaker,2,01:54-02:07
2,003,Happy,The Rainmaker,3,02:55-03:13
3,004,Happy,Man of Galilee CD1,2,03:02-03:18
4,005,Happy,Shakespeare in Love,1,01:06-01:22
5,006,Happy,The Rainmaker,5,03:07-03:23
6,007,Happy,Man of Galilee CD1,11,00:00-00:18
7,008,Happy,Nostradamus,1,00:00-00:16
8,009,Happy,Band of Brothers,3,01:19-01:36
9,010,Happy,The Untouchables,6,01:50-02:05


In [8]:
import librosa
import IPython.display as ipd

# Define the directory where your music files are located
music_dir = "SoundTracks/Set1/Set1"

# Function to load and play audio
def load_and_play_audio(row):
    # Extract the track number from the DataFrame
    track_number = row['Nro']
    
    # Construct the path to the audio file
    audio_path = f"{music_dir}/{track_number}.mp3"
    print(audio_path)
    # Load the audio file using librosa
    audio, sr = librosa.load(audio_path)
    
    # Display audio player
    ipd.display(ipd.Audio(audio, rate=sr))

# Apply the function to each row in the DataFrame
df.head(1).apply(load_and_play_audio, axis=1)


SoundTracks/Set1/Set1/001.mp3


0    None
dtype: object

In [11]:
import librosa
import numpy as np

def extract_features(file_path):
    x, sr = librosa.load(file_path)

    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=x, sr=sr, n_mfcc=20)

    # Take the mean along each feature dimension
    mfcc_mean = np.mean(mfcc, axis=1)

    features = mfcc_mean.reshape(1, 1, 20)  # Reshape to match the model input shape
    return features

In [22]:
from keras import backend as K
from keras.layers import Layer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")

        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # Applying attention mechanism
        et = K.squeeze(K.tanh(K.dot(x, self.W) + self.b), axis=-1)
        at = K.softmax(et)
        at = K.expand_dims(at, axis=-1)
        output = x * at

        # Summing the weighted inputs
        output = K.sum(output, axis=1)
        return output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [27]:
import tensorflow as tf
class GCNLayer(Layer):
    def __init__(self, output_dim):
        super(GCNLayer, self).__init__()
        self.output_dim = output_dim

    def build(self, input_shape):
        self.kernel = self.add_weight("kernel",
                                      shape=[int(input_shape[-1]), self.output_dim])

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)

In [29]:
import keras.backend as K

def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))

    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())

    f1_score = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_score

In [31]:
class_labels = np.array(['blues', 'classical', 'country', 'disco', 'hiphop', 'jazz',
                         'metal', 'pop', 'reggae', 'rock'])

In [30]:
from tensorflow.keras.models import load_model

model = load_model('BAG.h5', custom_objects={'AttentionLayer': AttentionLayer, 'GCNLayer': GCNLayer, 'f1_score': f1_score})

In [32]:
def predict_and_add_genre(row):
    # Extract the track number from the DataFrame
    track_number = row['Nro']
    
    # Construct the path to the audio file
    audio_path = f"{music_dir}/{track_number}.mp3"
    
    input_features = extract_features(audio_path)
    
    predictions = model.predict(input_features)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_label = class_labels[predicted_class_index]
    
    row['genre'] = predicted_class_label
    return row

In [33]:
df = df.apply(predict_and_add_genre, axis=1)

1/1 [==============================] - 0s 58ms/step


1/1 [==============================] - 0s 56ms/step


1/1 [==============================] - 0s 52ms/step


In [43]:
df.to_csv("SoundTracks/set1_updated_tracklist.csv", index=False)

In [99]:
def load_audio_for_emotion_and_genre(emotion, genre, df):
    filtered_df = df[(df['Emotion'] == emotion) & (df['genre'] == genre)]
    return filtered_df

In [100]:
file_path = "SoundTracks/set1_updated_tracklist.csv"

# Load CSV into DataFrame
df2 = pd.read_csv(file_path)
df2.head(2)

,Nro,Emotion,Album name,Track,Min:Sec,genre
0,1,Happy,Shallow Grave,6,02:02-02:17,jazz
1,2,Happy,The Rainmaker,2,01:54-02:07,classical


In [101]:
filtered_df = load_audio_for_emotion_and_genre(emotion='High Tens.', genre='country',df=df2)
filtered_df.head()

,Nro,Emotion,Album name,Track,Min:Sec,genre
305,306,High Tens.,Hellraiser,10,02:44-03:00,country
309,310,High Tens.,Last samurai,9,02:30-02:50,country
313,314,High Tens.,Batman,1,00:47-01:04,country
316,317,High Tens.,Angel Heart,3,00:43-00:59,country
323,324,High Tens.,Dracula,6,01:46-02:03,country


In [102]:
df['Emotion'].unique()

array(['Happy', 'Sad', 'Tender', 'Fear', 'Anger', 'Surprise', 'High Val.',
       'Low Val.', 'High Ener.', 'Low Ener.', 'High Tens.', 'Low Tens.'],
      dtype=object)

In [103]:
df2['genre'].unique()

array(['jazz', 'classical', 'country', 'blues', 'reggae', 'disco',
       'hiphop', 'pop', 'rock'], dtype=object)